In [1]:
import requests

In [5]:
HEADERS = {
    'Host': 'data.nba.net',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:79.0) Gecko/20100101 Firefox/79.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'api-key': 'PF2JvsskTXRs!rvmkU*.8YXg96DLKE3GN'
}

In [7]:
livescore = requests.get('https://fantasy.trashtalk.co/API/v1/livescore',
                                     headers=HEADERS,
                                     proxies=None).json()

In [23]:
livescore['scores']['category']['match'][1]

{'awayteam': {'id': '1171',
  'name': 'LA Clippers',
  'ot': '',
  'posession': 'False',
  'q1': '35',
  'q2': '21',
  'q3': '29',
  'q4': '18',
  'totalscore': '103'},
 'date': '20.10.2022',
 'datetime_utc': '21.10.2022 02:00',
 'formatted_date': '20.10.2022',
 'hometeam': {'id': '1066',
  'name': 'Los Angeles Lakers',
  'ot': '',
  'posession': 'False',
  'q1': '23',
  'q2': '33',
  'q3': '21',
  'q4': '20',
  'totalscore': '97'},
 'id': '232833',
 'player_stats': {'awayteam': {'bench': {'player': [{'assists': '0',
      'blocks': '0',
      'defense_rebounds': '4',
      'field_goals_attempts': '0',
      'field_goals_made': '0',
      'freethrows_goals_attempts': '2',
      'freethrows_goals_made': '1',
      'id': '3416',
      'minutes': '10',
      'name': 'N. Batum',
      'offence_rebounds': '1',
      'oncourt': 'False',
      'personal_fouls': '1',
      'plus_minus': '-5',
      'points': '1',
      'pos': 'PF',
      'steals': '0',
      'threepoint_goals_attempts': '0',
 